In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch


In [3]:
def get_data(idx):
    return [np.expand_dims(train_x[0][idx], axis=0), np.expand_dims(train_x[1].iloc[idx].values, axis=0)]

In [4]:
def make_gradcam_heatmap(inp, model, index):
    
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.layers[index].output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(inp)
        
    grads = tape.gradient(preds, last_conv_layer_output)

   
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))


    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output * pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy().sum(axis=-1).reshape(1, -1)

In [ ]:
def grad_cam(idx):
    _d = {0:"male", 1:"female"}
    pred = model.predict(get_data(idx))
    label = train_y[idx]
    print('pred:', pred, 'label:', label, '(label_type:', train.iloc[idx]['label_type'], ')')
    print('sex:', _d[train_x[1].iloc[idx]['sex']])
    print('age:', train_x[1].iloc[idx]['age'])
    lead_list = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    data = get_data(idx)
    heatmap = make_gradcam_heatmap(get_data(idx), model,  -7)
    
    for i in range(12):
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        ax.set_ylim([-1.5, 1.5])
        ax.set_xlim([0, 800])
        ax.set_title(lead_list[i])
        ax.plot(get_data(10)[0][0][:, i], color='b')
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        ax.imshow(heatmap, extent=[*xlim, *ylim], cmap='Reds', alpha=0.8, aspect=100)
        plt.grid(False)
        plt.show()
        9 